# 光学字符识别

![正在读报纸的机器人](./images/ocr.jpg)

检测和解释图像中的文本是一个常见的计算机视觉难题。此类过程通常被称为“*光学字符识别*”(OCR)。

## 使用计算机视觉服务读取图像中的文本

**计算机视觉**认知服务可为 OCR 任务提供支持，包括：

- 可用于读取多种语言的文本的 **OCR** API。此 API 可以同步使用，非常适合检测和读取图像中的少量文本。
- 针对较大文档进行了优化的**读取** API。此 API 以异步方式使用，可用于读取印刷体文本和手写文本。

可以通过创建**计算机视觉**资源或**认知服务**资源来使用此服务。

如果还没有创建资源，请在 Azure 订阅中创建**认知服务**资源。

> **备注**：如果已有认知服务资源，则只需在 Azure 门户中打开其“**快速入门**”页面，然后将其密钥和终结点复制到下面的单元格中即可。否则，请按照以下步骤创建认知服务资源。

1. 在另一个浏览器标签页中，打开 Azure 门户 (https://portal.azure.com) 并使用 Microsoft 帐户登录。

2. 单击“**&#65291;创建资源**”按钮，搜索“*认知服务*”并以如下设置创建**认知服务**资源：
    - **订阅**： *你的 Azure 订阅*。
    - **资源组**： *选择或创建具有唯一名称的资源组*。
    - **区域**： *选择任何可用区域*：
    - **名称**： *输入一个唯一名称*。
    - **定价层**：中的机器人 S0
    - **我确认我已阅读并理解上述通知**：已选中。
3. 等待部署完成。然后转到认知服务资源，并单击“**概述**”页面上的链接以管理该服务的密钥。你将需要使用终结点和密钥从客户端应用程序连接到认知服务资源。

### 获取认知服务资源的密钥和终结点

要使用认知服务资源，客户端应用程序需要其终结点和身份验证密钥：

1. 进入 Azure 门户，在认知服务资源的“**密钥和终结点**”页面上复制资源的“**Key1**”，并将其粘贴到以下代码中，替换“**YOUR_COG_KEY**”。
2. 复制资源的**终结点**，并将其粘贴到以下代码中，替换“**YOUR_COG_ENDPOINT**”。
3. 通过单击位于单元格左侧的“**运行单元格**”(&#9655;) 按钮运行下方单元格中的代码。

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_endpoint = 'YOUR_COG_ENDPOINT'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

现在你已设置好密钥和终结点，可以使用计算机视觉服务资源来提取图像中的文本。

我们先使用 **OCR** API，该 API 可用于同步分析图像并读取其中包含的所有文本。在本例中，我们使用虚构的 Northwind Traders 零售公司的广告图，该图中包含一些文本。运行下面的单元格来读取这些文本。 

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import os
%matplotlib inline

# Get a client for the computer vision service
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Read the image file
image_path = os.path.join('data', 'ocr', 'advert.jpg')
image_stream = open(image_path, "rb")

## Use the Computer Vision service to find text in the image
read_results = computervision_client.recognize_printed_text_in_stream(image_stream)

# Process the text line by line
for region in read_results.regions:
    for line in region.lines:

        # Read the words in the line of text
        line_text = ''
        for word in line.words:
            line_text += word.text + ' '
        print(line_text.rstrip())

# Open image to display it.
fig = plt.figure(figsize=(7, 7))
img = Image.open(image_path)
draw = ImageDraw.Draw(img)
plt.axis('off')
plt.imshow(img)

图像中的文本被整理为由区域、行和字词构成的层次结构，代码会读取这些内容以检索结果。

在结果中，查看在图像上读取到的文本。 

## 显示边界框

结果中还包括在图像中查找到的文本行和单独字词的*边界框*坐标。运行下面的单元格，查看上面检索的广告图中文本行的边界框。

In [ ]:
# Open image to display it.
fig = plt.figure(figsize=(7, 7))
img = Image.open(image_path)
draw = ImageDraw.Draw(img)

# Process the text line by line
for region in read_results.regions:
    for line in region.lines:

        # Show the position of the line of text
        l,t,w,h = list(map(int, line.bounding_box.split(',')))
        draw.rectangle(((l,t), (l+w, t+h)), outline='magenta', width=5)

        # Read the words in the line of text
        line_text = ''
        for word in line.words:
            line_text += word.text + ' '
        print(line_text.rstrip())

# Show the image with the text locations highlighted
plt.axis('off')
plt.imshow(img)

在结果中，每行文本的边界框都显示为图像上的矩形。

## 使用读取 API

前面使用的 OCR API 非常适合处理带有少量文本的图像。在需要读取较大篇幅的文本（例如扫描出的文档）时，则可以使用**读取** API。此过程需要完成多个步骤：

1. 向计算机视觉服务提交一个要以异步方式读取和分析的图像。
2. 等待分析操作完成。
3. 检索分析结果。

运行以下单元格，使用此过程读取发送给 Northwind Traders 商店经理的扫描信件中的文本。

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image
import time
import os
%matplotlib inline

# Read the image file
image_path = os.path.join('data', 'ocr', 'letter.jpg')
image_stream = open(image_path, "rb")

# Get a client for the computer vision service
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Submit a request to read printed text in the image and get the operation ID
read_operation = computervision_client.read_in_stream(image_stream,
                                                      raw=True)
operation_location = read_operation.headers["Operation-Location"]
operation_id = operation_location.split("/")[-1]

# Wait for the asynchronous operation to complete
while True:
    read_results = computervision_client.get_read_result(operation_id)
    if read_results.status not in [OperationStatusCodes.running]:
        break
    time.sleep(1)

# If the operation was successfuly, process the text line by line
if read_results.status == OperationStatusCodes.succeeded:
    for result in read_results.analyze_result.read_results:
        for line in result.lines:
            print(line.text)

# Open image and display it.
print('\n')
fig = plt.figure(figsize=(12,12))
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

查看结果。结果中完整转录出了信件内容，大部分内容都是印刷体文本，还有一个手写签名。信件的原始图像显示在 OCR 结果下方（可能需要滚动鼠标才能看到）。

## 读取手写文本

在前面的示例中，分析图像的请求指定了已针对读取*印刷*体文本进行了优化的文本识别模式。值得注意的是，尽管只是针对读取印刷体文本进行了优化，仍然读取到了手写签名。

这种读取手写文本的能力非常有用。例如，假设你写了一张包含购物清单的便条，并想使用手机上的应用来读取并转录其中包含的文本。

运行下面的单元格，查看手写购物清单的读取操作示例。

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image
import time
import os
%matplotlib inline

# Read the image file
image_path = os.path.join('data', 'ocr', 'note.jpg')
image_stream = open(image_path, "rb")

# Get a client for the computer vision service
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Submit a request to read printed text in the image and get the operation ID
read_operation = computervision_client.read_in_stream(image_stream,
                                                      raw=True)
operation_location = read_operation.headers["Operation-Location"]
operation_id = operation_location.split("/")[-1]

# Wait for the asynchronous operation to complete
while True:
    read_results = computervision_client.get_read_result(operation_id)
    if read_results.status not in [OperationStatusCodes.running]:
        break
    time.sleep(1)

# If the operation was successfuly, process the text line by line
if read_results.status == OperationStatusCodes.succeeded:
    for result in read_results.analyze_result.read_results:
        for line in result.lines:
            print(line.text)

# Open image and display it.
print('\n')
fig = plt.figure(figsize=(12,12))
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

## 更多信息

要详细了解如何使用计算机视觉服务完成 OCR，请参阅[计算机视觉文档](https://docs.microsoft.com/zh-cn/azure/cognitive-services/computer-vision/concept-recognizing-text)